# 03b. Train YOLOv8 Detector (Transfer Learning)

In [5]:
from ultralytics import YOLO
import os

## Configuration

In [6]:
# Paths
DATA_YAML = '../data/yolo_dataset/data.yaml'
MODEL_SAVE_DIR = '../models'

# Training parameters
EPOCHS = 20
IMG_SIZE = 640
BATCH_SIZE = 16  # Adjust based on your GPU memory
MODEL_SIZE = 'yolov8n'  # Options: yolov8n (nano), yolov8s (small), yolov8m (medium)

print(f'Model: {MODEL_SIZE}')
print(f'Epochs: {EPOCHS}')
print(f'Image Size: {IMG_SIZE}')
print(f'Batch Size: {BATCH_SIZE}')

Model: yolov8n
Epochs: 20
Image Size: 640
Batch Size: 16


## Load Pre-trained Model

In [ ]:
# Load pre-trained model
model = YOLO(f'{MODEL_SIZE}.pt')

print(f'Loaded pre-trained {MODEL_SIZE} model')
print(f'  This model was trained on COCO dataset (80 classes)')
print(f'  We will fine-tune it for license plate detection (1 class)')

✓ Loaded pre-trained yolov8n model
  This model was trained on COCO dataset (80 classes)
  We will fine-tune it for license plate detection (1 class)


## Train the Model

In [8]:
import torch

DEVICE = 0 if torch.cuda.is_available() else 'cpu'
print('Usando dispositivo:', DEVICE)

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    name='yolo_plate_detector',
    patience=10,
    save=True,
    device=DEVICE,
)


Usando dispositivo: 0
Ultralytics 8.3.229  Python-3.11.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_plate_detector11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=

## Evaluate the Model

In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# 1. Obtener las métricas del modelo
metrics = model.val(verbose=False)

# 2. Organizar los datos
data = {
    "Métrica": ["mAP@0.5 (Precisión IoU 0.5)", "mAP@0.5:0.95 (Robustez)", "Precision (Exactitud)", "Recall (Sensibilidad)"],
    "Valor": [metrics.box.map50, metrics.box.map, metrics.box.mp, metrics.box.mr],
    "Target (Meta)": ["> 0.90", "> 0.60", "> 0.90", "> 0.90"]
}

df_metrics = pd.DataFrame(data)

# --- OPCIÓN A: TABLA LIMPIA PARA EL NOTEBOOK ---
print("\n" + "="*40)
print("       RESULTADOS DE VALIDACIÓN")
print("="*40)
# Formatear valores a 4 decimales
print(df_metrics.to_markdown(index=False, floatfmt=".4f"))


# --- OPCIÓN B: GRÁFICA PARA LA PRESENTACIÓN ---
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Crear gráfico de barras
ax = sns.barplot(x="Valor", y="Métrica", data=df_metrics, palette="viridis")

# Línea de referencia de perfección (1.0)
plt.axvline(1.0, color='gray', linestyle='--', alpha=0.5)
plt.xlim(0, 1.1)

# Añadir los valores numéricos al lado de las barras
for i, v in enumerate(df_metrics["Valor"]):
    ax.text(v + 0.02, i, f"{v:.4f}", color='black', fontweight='bold', va='center')

plt.title("Rendimiento del Detector YOLOv8 (Validation Set)", fontsize=15, fontweight='bold')
plt.xlabel("Puntuación (0.0 - 1.0)", fontsize=12)
plt.ylabel("")

# Guardar para el reporte
plt.tight_layout()
plt.savefig('../reports/images/yolo_metrics.png', dpi=300)
print("\nGráfica guardada en reports/images/yolo_metrics.png")

plt.show()

Ultralytics 8.3.229  Python-3.11.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 1240.0492.5 MB/s, size: 353.6 KB)
val: Scanning C:\Users\Paco\Documents\github\Inteligencia_artificial\ProyectoFinal\data\yolo_dataset\labels\val.cache... 166 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 166/166  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 11/11 8.8it/s 1.2s<0.1s
                   all        166        166      0.956      0.924      0.981      0.657
Speed: 0.8ms preprocess, 3.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to C:\Users\Paco\Documents\github\Inteligencia_artificial\ProyectoFinal\notebooks\runs\detect\val5

       RESULTADOS DE VALIDACIÓN
| Métrica                     |   Valor | Target (Meta)   |
|:----------------------------|--------:|:----------------|
| mAP@0.5 (Precisión IoU 0.5) |  0.9810 | > 0.90         

<Figure size 1000x600 with 1 Axes>

## Save the Best Model

In [10]:
import os
import shutil

# Carpeta donde YOLO guarda los runs
runs_dir = os.path.join('runs', 'detect')
exp_prefix = 'yolo_plate_detector'

# Buscar todas las carpetas que empiezan por 'yolo_plate_detector'
run_dirs = []
if os.path.exists(runs_dir):
    run_dirs = [d for d in os.listdir(runs_dir)
                if d.startswith(exp_prefix) and os.path.isdir(os.path.join(runs_dir, d))]

if not run_dirs:
    print(f'No se encontraron runs en {runs_dir} con prefijo "{exp_prefix}"')
else:
    # Usar el último run (ordenado alfabéticamente: yolo_plate_detector, yolo_plate_detector2, ...7)
    last_run = sorted(run_dirs)[-1]
    best_model_path = os.path.join(runs_dir, last_run, 'weights', 'best.pt')
    destination = os.path.join(MODEL_SAVE_DIR, 'yolo_plate_detector.pt')

    if os.path.exists(best_model_path):
        os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
        shutil.copy(best_model_path, destination)
        print(f'Best model ({last_run}) saved to: {destination}')
    else:
        print(f'best.pt no encontrado en: {best_model_path}')


best.pt no encontrado en: runs\detect\yolo_plate_detector9\weights\best.pt


## Test on Sample Images

In [11]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Load the best model
best_model = YOLO(destination)

# Test on validation images
test_dir = '../data/yolo_dataset/images/val'
test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir)[:6]]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, img_path in enumerate(test_images):
    # Predict
    results = best_model(img_path)
    
    # Get annotated image
    annotated = results[0].plot()
    annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    axes[i].imshow(annotated)
    axes[i].set_title(os.path.basename(img_path))
    axes[i].axis('off')

plt.tight_layout()
plt.show()


image 1/1 c:\Users\Paco\Documents\github\Inteligencia_artificial\ProyectoFinal\notebooks\..\data\yolo_dataset\images\val\Cars10.png: 384x640 1 license_plate, 34.7ms
Speed: 1.9ms preprocess, 34.7ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 c:\Users\Paco\Documents\github\Inteligencia_artificial\ProyectoFinal\notebooks\..\data\yolo_dataset\images\val\Cars102.png: 512x640 1 license_plate, 31.5ms
Speed: 1.9ms preprocess, 31.5ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 c:\Users\Paco\Documents\github\Inteligencia_artificial\ProyectoFinal\notebooks\..\data\yolo_dataset\images\val\Cars104.png: 480x640 1 license_plate, 32.1ms
Speed: 1.2ms preprocess, 32.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\Paco\Documents\github\Inteligencia_artificial\ProyectoFinal\notebooks\..\data\yolo_dataset\images\val\Cars107.png: 384x640 1 license_plate, 5.8ms
Speed: 1.4ms preprocess, 5.8ms inference, 0

<Figure size 1500x1000 with 6 Axes>

## View Training Curves

Check `runs/detect/yolo_plate_detector/` for:
- `results.png` - Training/validation metrics
- `confusion_matrix.png` - Confusion matrix
- `val_batch*_pred.jpg` - Validation predictions

In [12]:
from IPython.display import Image, display
import os

runs_dir = os.path.join('runs', 'detect')
exp_prefix = 'yolo_plate_detector'

# Buscar runs que empiezan por yolo_plate_detector
run_dirs = []
if os.path.exists(runs_dir):
    run_dirs = [d for d in os.listdir(runs_dir)
                if d.startswith(exp_prefix) and os.path.isdir(os.path.join(runs_dir, d))]

if not run_dirs:
    print(f'No se encontraron runs en {runs_dir} con prefijo "{exp_prefix}"')
else:
    last_run = sorted(run_dirs)[-1]  # p.ej. yolo_plate_detector7
    results_img = os.path.join(runs_dir, last_run, 'results.png')
    if os.path.exists(results_img):
        display(Image(filename=results_img))
        print(f'Mostrando: {results_img}')
    else:
        print(f'Results image not found en: {results_img}')


Results image not found en: runs\detect\yolo_plate_detector9\results.png
